# Image Super Resolution

### Author: Slothfulwave612

In [5]:
## Listing out versions for each modules 
import sys
import keras
import cv2
import numpy
import matplotlib
import skimage

## modules versions
print('Python: {}'.format(sys.version))
print('Keras: {}'.format(keras.__version__))
print('OpenCV: {}'.format(cv2.__version__))
print('Numpy: {}'.format(numpy.__version__))
print('Matplotlib: {}'.format(matplotlib.__version__))
print('Skimage: {}'.format(skimage.__version__))

Python: 3.5.5 |Anaconda, Inc.| (default, Apr  7 2018, 04:52:34) [MSC v.1900 64 bit (AMD64)]
Keras: 2.2.4
OpenCV: 3.4.1
Numpy: 1.14.3
Matplotlib: 3.0.0
Skimage: 0.14.0


In [8]:
## import necessary packages
from keras.models import Sequential
from keras.layers import Conv2D, Input
from keras.optimizers import SGD, Adam
from skimage.measure import compare_ssim as ssim
import matplotlib.pyplot as plt
import numpy as np
import math
import os

## python magic function
%matplotlib inline

In [10]:
## function for peak signal-to-noise ratio (PSNR)
def psnr(target, ref):
    '''
    Function to produce low resolution image from high resolution image
    compare the two and calculate the score.
    
    Arguments:
    target -- numpy array, low resolution image.
    ref -- numpy array, high resolution image.
    
    Returns:
    float, score calculated using peak signal-to-noise ratio formula.
    '''
    target_data = target.astype(float)
    ref_data = ref.astype(float)
    
    diff = ref_data - target_data
    diff = diff.flatten(order='C')
    
    ## root mean squared error
    rmse = math.sqrt(np.mean(diff ** 2.))
    
    return 20 * math.log10(255. / rmse)      ## peak signal-to-noise ratio 

## function for mean squared error(MSE)
def mse(target, ref):
    '''
    Function to calculate mean squared error.
    
    Arguments:
    target -- numpy array, target image.
    ref -- numpy array, reference image.
    
    Returns:
    float, mean squared error value
    '''
    err = np.sum( (target.astype(float) - ref.astype(float)) ** 2 )
    err /= float(target.shape[0] * target.shape[1])
    
    return err

## function for comparing images
def compare_images(target, ref):
    '''
    Function that combines all three image quality metrics.
    Metrics:-   1. Peak Signal-to-Noise Ratio (PSNR)
                2. Mean Squared Error (MSE)
                3. Structural Similarity Index (SSIM)
                
    Arguments:
    target -- numpy array, a target image.
    ref -- numpy array, a reference image.
    
    Returns:
    scores -- list, containing the values for the three metrics.
    '''
    scores = []
    
    scores.append(psnr(target, ref))
    scores.append(mse(target, ref))
    scores.append(ssim(target, ref, multichannel=True))  
    
    return scores

In [13]:
def prepare_images(path, factor):
    '''
    Function to prepare degraded images by introducing distortions via resizing
    
    Arguments:
    path -- file path where the high-res images are stored.
    factor -- value by which resizing will take place.
    '''
    ## looping through the files in the directory
    for file in os.listdir(path):
        ## open the image
        img = cv2.imread(path + '/' + file)
        
        ## image dimention
        h, w, c = img.shape
        new_height = int(h / factor)
        new_width = int(w / factor)
        
        ## resizing the image - down
        img = cv2.resize(img, (new_width, new_height), interpolation=cv2.INTER_LINEAR)
        
        ## resizing the image - up
        img = cv2.resize(img, (w, h), interpolation=cv2.INTER_LINEAR)
        
        ## save the image
        cv2.imwrite('Images/{}'.format(file), img)
        print('Saving {}'.format(file))

In [15]:
## preparing the degraded images by calling the function
prepare_images('source', 4)

Saving baboon.bmp
Saving baby_GT.bmp
Saving barbara.bmp
Saving bird_GT.bmp
Saving butterfly_GT.bmp
Saving coastguard.bmp
Saving comic.bmp
Saving face.bmp
Saving flowers.bmp
Saving foreman.bmp
Saving head_GT.bmp
Saving lenna.bmp
Saving monarch.bmp
Saving pepper.bmp
Saving ppt3.bmp
Saving woman_GT.bmp
Saving zebra.bmp


In [17]:
## testing the generated images using image quality metrics
for file in os.listdir('images'):
    ## high-res images
    target = cv2.imread('source/{}'.format(file))
    
    ## low-res images
    ref = cv2.imread('Images/{}'.format(file))
    
    ## calculate the scores
    scores = compare_images(target, ref)
    
    ## display the results
    print('{}\nPSNR: {}\nMSE: {}\nSSIM: {}\n'.format(file, scores[0], scores[1], scores[2]))

baboon.bmp
PSNR: 19.792875018796664
MSE: 2046.0399708333334
SSIM: 0.4201422673399515

baby_GT.bmp
PSNR: 29.714944284952438
MSE: 208.30859375
SSIM: 0.8358366832703314

barbara.bmp
PSNR: 23.147023394416188
MSE: 945.1468171296297
SSIM: 0.666348651985364

bird_GT.bmp
PSNR: 27.31123529384467
MSE: 362.3081356095679
SSIM: 0.8509884459790468

butterfly_GT.bmp
PSNR: 20.21670148105035
MSE: 1855.80126953125
SSIM: 0.7163310820830953

coastguard.bmp
PSNR: 23.212548105536015
MSE: 930.993834832702
SSIM: 0.5041683329452455

comic.bmp
PSNR: 19.532668608947816
MSE: 2172.3747922437674
SSIM: 0.6005522476796749

face.bmp
PSNR: 28.51747878070106
MSE: 274.44399810964086
SSIM: 0.6920789883417745

flowers.bmp
PSNR: 23.21677884019166
MSE: 930.0873370165746
SSIM: 0.7103257877016409

foreman.bmp
PSNR: 25.348797624917474
MSE: 569.2746606691919
SSIM: 0.8428103511969228

head_GT.bmp
PSNR: 28.56528197512759
MSE: 271.43973214285717
SSIM: 0.6941511457749369

lenna.bmp
PSNR: 27.79536470199244
MSE: 324.08963775634766
SSI